## Data Cleaning Assignment

### Creating a data frame 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.DataFrame(
    {'From_To': 
     ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
     'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
     'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
     'Airline': 
     ['KLM(!)', '<Air France> (12)', '(British Airways. )','12. Air France', '"Swiss Air"']
    })
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


### Question 1
Flight missing numbers, fill, to integer type

In [3]:
for i in df.index:
    value = 10055 + (5 * (i))
    df.loc[i,'FlightNumber'] = value
    #print(value)
    
df['FlightNumber'] = df['FlightNumber'].astype(int)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
From_To         5 non-null object
FlightNumber    5 non-null int32
RecentDelays    5 non-null object
Airline         5 non-null object
dtypes: int32(1), object(3)
memory usage: 268.0+ bytes


In [5]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10055,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10060,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10070,[13],12. Air France
4,Brussels_londOn,10075,"[67, 32]","""Swiss Air"""


### Question 2
The From_To column would be better as two separate columns! Split each
string on the underscore delimiter _ to give a new temporary DataFrame with
the correct values. Assign the correct column names to this temporary
DataFrame.

## Method 1

In [6]:
# new_df = pd.DataFrame()

# new_df[['From', 'To']] = df['From_To'].str.split('_', expand=True)
# new_df

In [7]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10055,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10060,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10070,[13],12. Air France
4,Brussels_londOn,10075,"[67, 32]","""Swiss Air"""


## Method 2

In [8]:
new_df = pd.DataFrame()
df['From_To'] = df['From_To'].apply(lambda x: str(x).split('_'))

new_df['From'] = df['From_To'].map(lambda x : x[0])
new_df['To'] = df['From_To'].apply(lambda x: x[1])
new_df

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


### Question 3
Standardise the strings so that only the first letter is
uppercase 

In [9]:
new_df['From'] = new_df['From'].map(lambda x : str(x).title())
new_df['To'] = new_df['To'].map(lambda x : str(x).title())
new_df

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


### Question 4
Delete the From_To column from df and attach the temporary DataFrame
from the previous questions.

In [10]:
df.drop('From_To', axis=1, inplace=True)
df = pd.concat([df,new_df], axis=1)
df

,FlightNumber,RecentDelays,Airline,From,To
0,10055,"[23, 47]",KLM(!),London,Paris
1,10060,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10070,[13],12. Air France,Budapest,Paris
4,10075,"[67, 32]","""Swiss Air""",Brussels,London


### Question 5
In the RecentDelays column, the values have been entered into the
DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value
should be NaN.
Expand the Series of lists into a DataFrame named delays, rename the columns
delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df
with delays.

## Method 1

In [11]:
## Initializing a dictionary to hold the expanded series of list
new_dict = {}

## Max. no. of delays for each flight
max_len =  df['RecentDelays'].apply(len).max()

## Creating the rows of the delays
for i in df.index:
    row = []
    items = df.loc[i,'RecentDelays']
    row.extend(items)

    ## To maintain a uniform length for each row as no.of delays
    ## In absence of nth value, append it with None
    if len(row) < max_len:
        while len(row) < max_len:
            row.append(None)
    new_dict[i] = row

new_dict    

{0: [23, 47, None],
 1: [None, None, None],
 2: [24, 43, 87],
 3: [13, None, None],
 4: [67, 32, None]}

In [12]:
## delays Data Frame from the dictionary

delays = pd.DataFrame(new_dict)
delays = delays.T
delays

,0,1,2
0,23,47,NaN
1,None,None,None
2,24,43,87
3,13,NaN,NaN
4,67,32,NaN


In [13]:
## Giving proper names to the columns

col_name = ['delay' + str(i+1) for i in range(0,max_len)]

delays.columns = col_name
delays

,delay1,delay2,delay3
0,23,47,NaN
1,None,None,None
2,24,43,87
3,13,NaN,NaN
4,67,32,NaN


In [14]:
## Dropping the col 'RecentDelays' and adding the delays data frame

df.drop('RecentDelays',axis =1,inplace=True)

df = pd.concat([df,delays], axis = 1)
df

,FlightNumber,Airline,From,To,delay1,delay2,delay3
0,10055,KLM(!),London,Paris,23,47,NaN
1,10060,<Air France> (12),Madrid,Milan,None,None,None
2,10065,(British Airways. ),London,Stockholm,24,43,87
3,10070,12. Air France,Budapest,Paris,13,NaN,NaN
4,10075,"""Swiss Air""",Brussels,London,67,32,NaN


In [15]:
df.rename(columns={'delay1':'delay_1','delay2':'delay_2','delay3':'delay_3'}, inplace=True)
df.columns

Index(['FlightNumber', 'Airline', 'From', 'To', 'delay_1', 'delay_2',
       'delay_3'],
      dtype='object')

In [16]:
df

,FlightNumber,Airline,From,To,delay_1,delay_2,delay_3
0,10055,KLM(!),London,Paris,23,47,NaN
1,10060,<Air France> (12),Madrid,Milan,None,None,None
2,10065,(British Airways. ),London,Stockholm,24,43,87
3,10070,12. Air France,Budapest,Paris,13,NaN,NaN
4,10075,"""Swiss Air""",Brussels,London,67,32,NaN


## Method 2

In [17]:
# delays = pd.DataFrame(df['RecentDelays'].to_list(), columns=['delay_1','delay_2','delay_3'])
# delays

In [18]:
df

,FlightNumber,Airline,From,To,delay_1,delay_2,delay_3
0,10055,KLM(!),London,Paris,23,47,NaN
1,10060,<Air France> (12),Madrid,Milan,None,None,None
2,10065,(British Airways. ),London,Stockholm,24,43,87
3,10070,12. Air France,Budapest,Paris,13,NaN,NaN
4,10075,"""Swiss Air""",Brussels,London,67,32,NaN
